## Encoder を 15 レイヤー用意し，時系列的に入力するモデルの検証


### Import


In [29]:
import python.util as util
from pathlib import Path
import importlib
from typing import Dict
from transformers import RobertaForSequenceClassification
import torch
from datasets import Dataset
import pandas as pd
import numpy as np
import yaml

### DEFINE PATH


In [30]:
BASE_PATH = Path.cwd().parent
DATA_PATH = BASE_PATH / "data"
MODEL_PATH = BASE_PATH / "model"
print(f"BASE_PATH: {BASE_PATH}")
print(f"DATA_PATH: {DATA_PATH}")
print(f"MODEL_PATH: {MODEL_PATH}")

BASE_PATH: /home/masa1357/Dockerdata/gitfile/15-Encoders
DATA_PATH: /home/masa1357/Dockerdata/gitfile/15-Encoders/data
MODEL_PATH: /home/masa1357/Dockerdata/gitfile/15-Encoders/model


### Set Seed & Set Logger


In [31]:
importlib.reload(util)
# ? logger読み込み
name = "15-Ensemble"
logger = util.set_logger()
# ? seed値固定
seed = 42
util.set_seed(seed)

2024-09-28 06:15:48,462 : python.util : INFO : 34 : Test_message


In [32]:
# ? データ読み込み
import python.data_manager as dm

importlib.reload(dm)
# argsでloggerの名前を送る

args: Dict[str, object] = {
    "logger": logger,
    "DATA_PATH": DATA_PATH,
    "key": "userid",
    "text": "answer_content",
    "label": "grade",
    "mode": "all",
    "split_rate": [0.8, 0.2, 0],
    "seed": seed,
}

with util.timer("load data"):
    dp = dm.DataProcessing(args)
    df = dp()

2024-09-28 06:15:48,489 : python.util.python.data_manager : INFO : 16 : setup Child logger : python.data_manager
2024-09-28 06:15:48,491 : python.util.python.data_manager : INFO : 65 : Read /home/masa1357/Dockerdata/gitfile/15-Encoders/data/Reflection *.csv data...
2024-09-28 06:15:48,492 : python.util.python.data_manager : INFO : 67 : Found files: [PosixPath('/home/masa1357/Dockerdata/gitfile/15-Encoders/data/Reflection/Course_C-2021-2_Reflection.csv'), PosixPath('/home/masa1357/Dockerdata/gitfile/15-Encoders/data/Reflection/Course_C-2022-1_Reflection.csv'), PosixPath('/home/masa1357/Dockerdata/gitfile/15-Encoders/data/Reflection/Course_C-2021-1_Reflection.csv')]
2024-09-28 06:15:48,611 : python.util.python.data_manager : INFO : 74 : Total rows: 28275
2024-09-28 06:15:48,612 : python.util.python.data_manager : INFO : 65 : Read /home/masa1357/Dockerdata/gitfile/15-Encoders/data/Grade *.csv data...
2024-09-28 06:15:48,614 : python.util.python.data_manager : INFO : 67 : Found files: [Pos

[load data] start


2024-09-28 06:15:48,650 : python.util.python.data_manager : INFO : 146 : dump_df shape : (7984, 6)


[load data] done in 0.73 s


In [33]:
df.head(3)

,userid,grade,text,course_number,question_number,label
0,C-2021-1_U1,F,情報伝達には声モルス信号など様々な形があり人類の進化と共により高度に精度が高められてきた,01,1,4
1,C-2021-1_U10,B,高校時代で習った情報だけでは現代の進化についていけない\r\n情報はどんな形であっても〇と●...,01,1,1
2,C-2021-1_U100,C,情報の発展の仕方\r\n授業の参加の仕方,01,1,2


### 各生徒の回答を時系列でグループ化


In [34]:
# 質問番号でフィルタ
filter = "1"
Qfilter_df = dm.encode(df, logger=logger, filter=filter)
Qfilter_df.head(3)

/home/masa1357/Dockerdata/gitfile/15-Encoders/src/python/data_manager.py:280: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "new_column"] = df.apply(create_column_name, axis=1)


new_column,userid,label,01-1,02-1,03-1,04-1,05-1,06-1,07-1,08-1,09-1,10-1,11-1,12-1,13-1,14-1,15-1
0,C-2021-1_U1,4,情報伝達には声モルス信号など様々な形があり人類の進化と共により高度に精度が高められてきた,情報源符号化とは情報を黒と白の○で表すことで複合とは符号化によって得られたものを元に戻す作業...,生起確率pの事象の生起を知ったことによる曖昧さの減少量は情報量と一致し生起確率pが高いほど情...,通信の際に符号の誤りを検出・訂正する仕組みについて学んだ符号の誤りは符号語同士がs+1以上離...,コンピュタサイエンスは計算機科学であるともされ全ての科学技術分野の基盤となりうるものであるコ...,計算ステップ数とはアルゴリズムが終了するまでに行われる演算の数である最大公約数の問題では剰余...,マジソトとは入力数列を分割していき分割したものを整列して統合(マジ)するアルゴリズムであるヒ...,[NA],[NA],[NA],[NA],[NA],[NA],[NA],[NA]
1,C-2021-1_U10,1,高校時代で習った情報だけでは現代の進化についていけない\r\n情報はどんな形であっても〇と●...,どのように情報を短く表現するか\r\n符号をを一意に素早く戻すためにはどのような符号にすればよいか,情報の量を表すためにはどのようにすればよいか\r\n2つの現象が起こった場合相互の情報量はど...,誤りの自動検出と自動訂正が可能なハミング距離\r\n符号化効率とブロック誤り率,基礎科学としてのコンピュタサイエンス\r\nコンピュタサイエンスにおける計算と問題とは何か\...,色々なソトの種類とその計算時間量や比較回数\r\n,マジソトと2分探索法\r\n比較ベスのソトアルゴリズムのまとめ,デタとは何か\r\nデタの予測発見分類・グルピング\r\n,人工知能(AI)特化型AIとは何か\r\n身の回りの人工知能\r\n人工知能の機械学習\r\...,非構造化デタの処理\r\nパタン認識とは何か\r\nパタン認識と機械学習,デタ収集とバイアス\r\n個人情報とデタ\r\nオプンデタについて,ベクトルによるデタ表現ベクトルとは何か\r\nベクトルとデタ分析の関係\r\n距離や類似度と...,デタの可視化とは\r\n色々な可視化方法\r\n多次元デタの可視化,相関デタの広がりと分散\r\n統計的検定とは何か\r\n画像処理と解析(フィルタ処理),期末テスト\r\n授業アンケト
2,C-2021-1_U100,2,情報の発展の仕方\r\n授業の参加の仕方,情報源符号化によって情報が伝えられるときより軽く瞬時に伝わるような仕組み,エントロピ情報量について,通信路符号化ビット反転によってノイズが生じる仕組み自動訂正自動検出について,計算とアルゴリズムについて効率のよい計算,いろいろなソトについて,マジソト2分探索法について,デタについて,AIについての基礎情報,非構造デタとはパタン認識とは,デタの収集にはどのような方法があるのか調査方法バイアスとは個人情報について,ベクトルについていろいろな距離についてデタにおける類似とは,デタの可視化について,デタの間の関係性相関統計的検定について画像解析フィルタ処理について,期末テスト


### Convert to Dataset


In [35]:
# dataset化
# datasetの概要：train/validの2種類
# train: 0.8, valid: 0.2
# input:15個のの回答文ー＞時系列情報を持っているため，dfの列名に含まれている番号を考慮して順番を守って保持
# label: df['label']を利用
# 

def create_dataset(df: pd.DataFrame, label_column: str):
    # データフレームからinputデータとlabelを抽出
    inputs = df[[f"{i:02}-1" for i in range(1, 16)]].values.tolist()
    labels = df[label_column].tolist()
    
    # データセットの形式に変換
    dataset_dict = {
        'input': inputs,
        'label': labels
    }
    
    # Dataset型に変換
    dataset = Dataset.from_dict(dataset_dict)
    
    return dataset

In [36]:
train_dataset = create_dataset(Qfilter_df, 'label')

In [37]:
# train_datasetの確認
display(train_dataset[0])
# train_datasetの形状確認
display("train_datasetの形状", train_dataset[0].keys())

# train_datasetの行数確認
display("train_datasetの行数", len(train_dataset))


{'input': ['情報伝達には声モルス信号など様々な形があり人類の進化と共により高度に精度が高められてきた',
  '情報源符号化とは情報を黒と白の○で表すことで複合とは符号化によって得られたものを元に戻す作業のことであるそれらの過程の中で平均符号語長を短くすることが目指されるそこで一意符号可能性と瞬時復号可能性を持つ語頭符号を用いる\r\nエントロピとは平均符号語長の下限のことである',
  '生起確率pの事象の生起を知ったことによる曖昧さの減少量は情報量と一致し生起確率pが高いほど情報量は現状しまた逆もしかりであるまたエントロピは情報量の期待値と一致する相互情報量とはある事象2つが相互に関連して生起した場合に得られる情報量のことである',
  '通信の際に符号の誤りを検出・訂正する仕組みについて学んだ符号の誤りは符号語同士がs+1以上離れていればs個検出でき2t+1離れていればt個訂正することができるまたその距離をハミング距離と言う',
  'コンピュタサイエンスは計算機科学であるともされ全ての科学技術分野の基盤となりうるものであるコンピュタ上で行われている全てのことは計算でありコンピュタサイエンスはそれを制御・理解するための科学である\r\nアルゴリズムとは関数の計算方法を示す手続きのことを言いプログラムとはアルゴリズムをコンピュタが分かる言葉で記述したものであるアルゴリズムを上手く考えることができれば偽コイン発見の問題のように作業を効率化最適化することができる大きな数の因数分解は難しいためユクリッドの互除法を用いれば容易に計算できる',
  '計算ステップ数とはアルゴリズムが終了するまでに行われる演算の数である最大公約数の問題では剰余を考えるバブルソトとは隣り合う要素の大小を比較しながら昇順もしくは降順に整列していくことである2進木のソトを読むときには通りがけ順で左の子→自分→右の子の順番で読む２進木の一種にヒプがある',
  'マジソトとは入力数列を分割していき分割したものを整列して統合(マジ)するアルゴリズムであるヒプソトと同様にO(nlogn)回の比較で要素を整列する2進木を使用しないためよりシンプルである最悪時の比較回数は決定木の高さに等しい\r\n線形探索が数列を先頭から探索していくのに対し二部探索は真ん中の要素から比較して前半部分・後半部分

'train_datasetの形状'

dict_keys(['input', 'label'])

'train_datasetの行数'

374

### Load Architecture Params


In [38]:
with open("./config/Encoder_Architecture.yaml") as file:
    EncoderModelConfig = yaml.safe_load(file)
    logger.info(EncoderModelConfig)

2024-09-28 06:15:49,507 : python.util : INFO : 3 : {'num_layers': 15, 'd_model': 512, 'num_heads': 8, 'dff': 2048, 'dropout_rate': 0.1}


### Load Encoder Model Weight


In [39]:
# Load Encoder model weight
name = "roberta_base_japanese_default"
if not (MODEL_PATH / name).exists():
    logger.warning(f"Model weight not found: {name}")
    logger.info("load online model weight")
    model = RobertaForSequenceClassification.from_pretrained(
        "nlp-waseda/roberta-base-japanese", num_labels=5
    )
    # Save model
    model.save_pretrained(MODEL_PATH / "roberta_base_japanese_default")
else:
    logger.info(f"Model weight found: {name}")
    model = RobertaForSequenceClassification.from_pretrained(
        MODEL_PATH / name, num_labels=5
    )

2024-09-28 06:15:49,514 : python.util : INFO : 12 : Model weight found: roberta_base_japanese_default


In [40]:
# モデルの確認
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [41]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Something is happening before the function is called.")
        result = func(*args, **kwargs)
        print("Something is happening after the function is called.")
        return result
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.
